# Summarization Using Pegasus Model

## This notebook outlines the concepts behind finetuning a Summarization model using T-5 variant model

In [1]:
!pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-2g8ht91m
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-2g8ht91m
  Resolved https://github.com/huggingface/accelerate to commit eba6eb79dc2ab652cd8b44b37165a4852768a8ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=226907 sha256=b79fe5edcf2eb8a892046ec8bdf25b0176265a6a0d22f01df6371478ae021219
  Stored in directory: /tmp/pip-ephem-wheel-cache-wv4pypqb/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


## Import Necessary Libraries

In [2]:
! pip install -q datasets transformers rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.5 MB/s eta 0:00:00


In [3]:
import torch
torch.cuda.empty_cache()
from datasets import load_dataset, load_metric
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

## Loading the dataset

link: https://www.kaggle.com/datasets/sunnysai12345/news-summary <br>
take news summary more file

In [6]:
data = pd.read_csv('data/news_summary_more.csv',nrows=10000)

In [7]:
# Split the data into train and test sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Split the train set further into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [8]:
# Create a Dataset object for each split
train_dataset = Dataset.from_dict(train_df)
val_dataset = Dataset.from_dict(val_df)
test_dataset = Dataset.from_dict(test_df)

To access an actual element, you need to select a split first, then give an index:

In [9]:
# Create a DatasetDict object with the splits
data = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['headlines', 'text'],
        num_rows: 6400
    })
    validation: Dataset({
        features: ['headlines', 'text'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['headlines', 'text'],
        num_rows: 2000
    })
})

###  To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [11]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(data["train"])

,headlines,text
0,Sheikh Hasina secures third straight term as Bangladesh PM,"Bangladesh PM Sheikh Hasina has secured her third consecutive term, with her alliance winning 287 of the 298 seats for which results have been declared for the 300-strong parliament, the country's Election Commission said on Monday. The main opposition Bangladesh Nationalist Party, which boycotted the 2014 poll, has won just six seats in the general election held on Sunday."
1,BSE creates new division for listing startups,"The Bombay Stock Exchange (BSE) on Wednesday announced the creation of a new division within its small and medium enterprises (SMEs) segment to list startups. Startups seeking listing should have existed for at least two years at the time of filing. Further, they should not have been referred to National Company Law Tribunal (NCLT) under Insolvency and Bankruptcy Code."
2,General category quota will get NDA 10% more votes: Paswan,Union Minister Ram Vilas Paswan on Sunday said that the measure for 10% general category quota will boost the BJP-led NDA's vote share by 10%. He asserted people will reject the opposition's proposed grand alliance due to its inherent contradiction and instability. He further said Dalits now know that PM Narendra Modi is not anti-Dalit as used to be projected.
3,855 police officers awarded medals on eve of Republic Day,"As many as 855 police personnel were awarded medals on the eve of Republic Day by President Ram Nath Kovind. Of these, 146 were awarded Police Medals for Gallantry (PMG) and three personnel were decorated with the Presidentâs Police Medal for Gallantry (PPMG) award. CRPF was decorated with maximum PMG, 41, followed by Odisha Police which received 26 medals."
4,Saw 'Andaz Apna...' to erase 'The Conjuring' memories: Vicky,"Vicky Kaushal has revealed he can't watch horror films, adding, ""I have watched 'The Conjuring'...I had to watch 'Andaz Apna Apna' right after it to erase all the memories of 'The Conjuring'...and that's when I could...sleep."" Vicky revealed this while interacting on Neha Dhupia's show 'No Filter Neha'. He also spoke about his rumoured girlfriend Harleen Sethi on the show."


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [13]:
metric = load_metric("rouge")
metric

<ipython-input-13-e6735d292d7d>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [14]:
data['train'][0]

{'headlines': "Sushant to star in Bhandarkar's 'Inspector Ghalib': Reports",
 'text': 'Sushant Singh Rajput will be starring in Madhur Bhandarkar\'s next film based on sand mafias titled \'Inspector Ghalib\', as per reports. The story, which is inspired by real-life events, is based in Uttar Pradesh and the major part of the film will be shot there, reports suggested. "\'Inspector Ghalib\' is the story of a cop," stated reports.'}

In [15]:
model_checkpoint = "google/bigbird-pegasus-large-arxiv"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## Preprocessing the data

In [16]:
max_input_length = 1024
max_target_length = 128
prefix = 'summarize'
def preprocess_function(examples):
  
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["headlines"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
preprocess_function(data['train'][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[24710, 20159, 59206, 8949, 73524, 138, 129, 11692, 115, 68025, 551, 88689, 10310, 131, 116, 352, 896, 451, 124, 3391, 59803, 116, 6486, 1034, 87460, 38521, 11656, 131, 108, 130, 446, 1574, 107, 139, 584, 108, 162, 117, 2261, 141, 440, 121, 4527, 702, 108, 117, 451, 115, 27652, 12118, 111, 109, 698, 297, 113, 109, 896, 138, 129, 1785, 186, 108, 1574, 3498, 107, 198, 131, 87460, 38521, 11656, 131, 117, 109, 584, 113, 114, 17934, 745, 3163, 1574, 107, 1], [24710, 51208, 2687, 4037, 1271, 86269, 596, 25451, 108, 464, 2901, 114, 437, 140, 3252, 115, 1307, 118, 10431, 120, 49114, 135, 169, 2741, 19556, 25950, 108, 148, 174, 4571, 10168, 107, 596, 25451, 140, 4571, 10168, 124, 114, 510, 4517, 113, 110, 105, 4363, 55778, 198, 19564, 667, 2097, 6242, 635, 109, 2299, 135, 213, 401, 2420, 652, 170, 134, 326, 161, 2741, 131, 116, 49114, 1422, 2755, 112, 7929, 745, 109, 11872, 196, 4620, 333, 114, 481, 8418, 107, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [18]:
tokenized_datasets = data.map(preprocess_function, batched=True)

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Importing Pretrained Model and Tokenizer

In [19]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

## Setting Up Arguments of Model for Fine Tuning

In [20]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

## Define Compute Metrics

In [22]:
import nltk
import numpy as np
nltk.download('punkt')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Initialize `Seq2SeqTrainer` for training model on custom dataset:

In [23]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [24]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Attention type 'block_sparse' is not possible if sequence_length: 83 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,5.178900,4.911960,32.296300,12.642000,29.163100,29.141500,15.444400


TrainOutput(global_step=1600, training_loss=5.6054714584350585, metrics={'train_runtime': 1137.9432, 'train_samples_per_second': 5.624, 'train_steps_per_second': 1.406, 'total_flos': 1528417930346496.0, 'train_loss': 5.6054714584350585, 'epoch': 1.0})

## Pass Sample text to generate summary of model

In [26]:
sample_text = "Two years ago, swing and seam proved to be India’s bugbears in the World Test Championship (WTC) final against New Zealand. The four-pronged pace attack of Tim Southee, Trent Boult, Neil Wagner and Kyle Jamieson had left the Indians on the mat. In a couple of days time, the Rohit Sharma-led team will again be up against the likes of Pat Cummins and Mitchell Starc, who nonetheless are a notch better than their trans-Tasman rivals. The age-old manual of how to bat in England is simple. Wait. But the Indians don’t do waiting, all that well. Especially when the ball swings around. The swing breaks their soul and about the time the ball starts to curve away or shape in, Indian batsman start to freeze. The balance starts to go topsy-turvy, the bat follows and there they break the thumb rule of batsmanship: Never play away from your body. It will be familiar territory for the Indian batsmen with the trajectory of the swinging ball winking at them. Coming right after the IPL, the lack of preparation might hinder India’s chance to end their ICC title drought."
encoded_input = tokenizer(sample_text, truncation=True, padding=True, max_length=512, return_tensors="pt")


In [27]:
device = 'cpu'

In [28]:
# Move the model to the same device as the input tensors
model = model.to(device)

In [29]:
input_ids = encoded_input["input_ids"].to(device)
attention_mask = encoded_input["attention_mask"].to(device)

In [30]:
# Generate the summary
output = model.generate(input_ids=input_ids, attention_mask=attention_mask)
generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (256) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [31]:
generated_summary

'swing and seam are bugbears in the final'